In [2]:
import ee
import geemap
import json
import os
import google.auth
import google.auth.transport.requests
from google.oauth2.credentials import Credentials

# Authenticate and initialize Earth Engine
if 'EARTHENGINE_TOKEN' in os.environ:
    # Running on Heroku: Use the token from the environment variable
    credentials_data = json.loads(os.environ['EARTHENGINE_TOKEN'])
    
    credentials = Credentials(
        None,
        refresh_token=credentials_data["refresh_token"],
        token_uri="https://oauth2.googleapis.com/token",
        client_id=credentials_data["client_id"],
        client_secret=credentials_data["client_secret"],
        scopes=credentials_data["scopes"]
    )
    
    request = google.auth.transport.requests.Request()
    credentials.refresh(request)
    ee.Initialize(credentials)
else:
    # Running locally: Ensure you have authenticated using `earthengine authenticate`
    # and specify your project if necessary
    ee.Initialize(project='ee1-seanvdahlman')  # Replace with your project ID

# Create a map instance centered on the study area
Map = geemap.Map(center=[28.6532, 77.156], zoom=10)

# Load built-up surface images for 1975 and 2020
image_1975 = ee.Image('JRC/GHSL/P2023A/GHS_BUILT_S/1975').select('built_surface')
image_2020 = ee.Image('JRC/GHSL/P2023A/GHS_BUILT_S/2020').select('built_surface')

# Create an image collection with indexed years
collection = ee.ImageCollection([
    image_1975.set('system:index', '1975'),
    image_2020.set('system:index', '2020')
])

# Labels for the time slider
labels = ['1975', '2020']

# Define visualization parameters for the images
visParams = {
    'min': 0.0,
    'max': 8000.0,  # Adjusted max to match dataset range
    'palette': ['000000', 'FFFFFF']  # Adjusted colors for better visualization
}

# Apply visualization parameters using the time slider inspector
try:
    Map.ts_inspector(
        left_ts=collection.map(lambda img: img.visualize(**visParams)), 
        right_ts=collection.map(lambda img: img.visualize(**visParams)), 
        left_names=labels, 
        right_names=labels
    )
except AttributeError as e:
    print("It appears your geemap version might not support ts_inspector. Consider updating geemap using 'pip install --upgrade geemap'.")

# Create and add a legend to the map
legend_dict = {
    'No Built Surface (0)': '#000000',
    'Max Built Surface (8000)': '#FFFFFF'
}
Map.add_legend(title='Built Surface', legend_dict=legend_dict)

# Add additional controls: scale bar and north arrow
import ipyleaflet
from ipywidgets import HTML

# 1. Add a scale bar (default is bottom-left)
scale_control = ipyleaflet.ScaleControl(position='bottomleft')
Map.add_control(scale_control)

# 2. Add a north arrow
north_arrow_html = HTML('''
    <div style="text-align: center;">
        <img src="https://upload.wikimedia.org/wikipedia/commons/8/8e/North.svg" 
             style="width:40px; transform: rotate(0deg);" 
             alt="North Arrow" />
    </div>
''')
north_arrow_control = ipyleaflet.WidgetControl(widget=north_arrow_html, position='topright')
Map.add_control(north_arrow_control)

# Display the map (make sure this is the last line in your cell)
Map


Map(center=[28.6532, 77.156], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=Dropdown(…